In [2]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X) 

tensor([[ 0.1547,  0.0897,  0.1649,  0.0013,  0.1421, -0.0092, -0.0526,  0.1932,
          0.1911, -0.0037],
        [ 0.0950,  0.0010,  0.2089,  0.0334, -0.0872,  0.0671, -0.1347,  0.0544,
          0.0844,  0.1611]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # Hidden layer(fully-connected)
        self.out = nn.Linear(256, 10) # Output layer(fully-connected)
    
    # how to output based on the input
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP() 
net(X)

tensor([[ 0.0525, -0.0133,  0.0085, -0.1347,  0.0805,  0.2007,  0.0121,  0.1423,
          0.0775,  0.0590],
        [ 0.1742, -0.0619, -0.0534, -0.0850,  0.0560,  0.1060, -0.1070,  0.2264,
          0.1109,  0.0877]], grad_fn=<AddmmBackward0>)

In [4]:
# how does Sequential works 
class mySequential(nn.Module): 
    def __init__(self, *args): 
        super().__init__() 
        for idx, module in enumerate(args): 
            # register the passed-in layer into submodule ordered dict of current model
            self._modules[str(idx)] = module 

    def forward(self, X): 
        # ordered list ensure the traverse sequence follows the sequence which we add it
        for block in self._modules.values(): 
            X = block(X) 
        return X

# accept a 20-dim(size) vector and transform into a vector of size 256
# ReLU turn all the negative numbers to 0 (given the ability to fit a non-linear graph)
# accept a 256-dim features and map them down to 10
net = mySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10)) 
net(X)

tensor([[-0.0897,  0.2411, -0.3453, -0.0343, -0.2121, -0.1365, -0.1697,  0.0139,
         -0.0314,  0.3589],
        [-0.0339,  0.1304, -0.2281,  0.0861, -0.1791, -0.1028, -0.1966, -0.0430,
         -0.0651,  0.1669]], grad_fn=<AddmmBackward0>)

In [5]:
# A more flexible way to define a model
import torch
from torch import nn
from torch.nn import functional as F 

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # random weight no need to calculate its grad
        # define a 20*20 tensor with random weight （this W is fixed）
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        # define an input-layer, it will automatically define W & b, these param will keep optimized while training
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # defining a fully-connected layer(Linear layer) (Y = X·W + b)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # reuse the same linear layer
        X = self.linear(X)

        # Ensure L1(norm) is <= 1 (avoiding grad explosion)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum() 

net = FixedHiddenMLP()
net(X)

tensor(0.1696, grad_fn=<SumBackward0>)

In [8]:
# Modular nesting
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # (mini MLP)20-dim to 64-dim to 32-dim to 16-dim
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16) 
        
    def forward(self, X):
        return self.linear(self.net(X))

# because the fixed W in FixedHiddenMLP is 20*20 , so before feeding the output to the last model
# we need to transform to 20-dim
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0851, grad_fn=<SumBackward0>)

# 2. Params management

#### A. access the data

In [11]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
print(net(X)) 

# Access the third layer (index 2) and print its parameters
# .state_dict() tells the weight & bias of the current layer
print(net[2].state_dict())

tensor([[-0.1894],
        [-0.1894]], grad_fn=<AddmmBackward0>)
OrderedDict([('weight', tensor([[ 0.3225, -0.1955,  0.2998, -0.1394,  0.0646, -0.1265,  0.0890,  0.1692]])), ('bias', tensor([-0.1894]))])


In [19]:
# Now we know that the param is not just a tensor, 
# but an instance of the nn.Parameter class.print(type(net[2].bias))
print(type(net[2].bias))
print('\n')

print(net[2].bias) # this will tell us about the  and is grad included
print('\n')

print(net[2].bias.data) # this will only tell us about the raw tensor data

<class 'torch.nn.parameter.Parameter'>


Parameter containing:
tensor([-0.1894], requires_grad=True)


tensor([-0.1894])


In [21]:
net[2].weight.grad == None # chk if the grad of third layer's weight is empty

True

In [26]:
# chk the weight and bias of the first linear layer
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

# chk the weight & bias of each layer
# net[0] hidden layer （4 inputs to 8 hidden units）
# net[1] ReLU(activate function of hidden layer)
# net[2] output layer
print(*[(name, param.shape) for name, param in net.named_parameters()]) 

print(net.state_dict()['2.bias'].data)

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
tensor([-0.1894])


In [28]:
def block1(): 
    # def a function: pick the input frm 4-dim to 8-dim to 4-dim 
    # 4 to 8: is like looking closer to pick the high-dim features to learn 
    # 8 to 4: is like concluding what the model will learn 
    # there're ReLU() at the end of these 2 layers to make sure it can handle complex curves 
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,4), nn.ReLU())

def block2(): 
    # now we start to define a model
    # By splicing block1 4 times, we can get a model with 4*2 = 8 linear layers
    net = nn.Sequential() 
    for i in range(4): 
        net.add_module(f'block {i}', block1()) 
    return net 

rgnet = nn.Sequential(block2(), nn.Linear(4,1)) 
print(rgnet(X)) 
print(rgnet) # to know what happen in each layers

tensor([[0.0248],
        [0.0248]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [30]:
# access the bias.data of first layer[0] of second sub0-block[1] of first big block[0], 
print(rgnet[0][1][0].bias.data) 

tensor([ 0.4623,  0.3198, -0.1012,  0.2696, -0.2987,  0.3460, -0.2654, -0.4008])


#### B. Initialize the data

In [34]:
def init_normal(m):
    if type(m) == nn.Linear:
        # replace the value of the weight matrix to a random num that fits a normal distibution with mean of 0 and std deviation of 0.01
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias) 
net.apply(init_normal)
print(net[0].weight.data[0], net[0].bias.data[0])


def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)   
net.apply(init_constant)
print(net[0].weight.data[0], net[0].bias.data[0])

tensor([ 0.0113, -0.0150, -0.0047,  0.0043]) tensor(0.)
tensor([1., 1., 1., 1.]) tensor(0.)


In [36]:
# Xavier initialize: Keep input, output, and gradient variances consistent across all layers.
# A great method to prevent grad vanish or explode
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) 
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)
print(net)

tensor([ 0.0768, -0.6218,  0.4811, -0.1065])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [39]:
# Customize initialization
# [-10,10]: 10-(-10) = 20, in (-5,5): 5-(5) = 10, 10/20 = 1/2 
# [5,10]: 10-5 = 5, 5/20 = 1/4 
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        
        # uniform_: pick a random num locally frm [-10,10]
        nn.init.uniform_(m.weight,-10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 # abs <= 5, *0; else *1

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000, -5.1336, -9.8545],
        [-0.0000, -0.0000, -7.5281, -9.5152]], grad_fn=<SliceBackward0>)

$$
w \sim \begin{cases} 
U(5, 10) & \text{可能性 } \frac{1}{4} \\ 
0 & \text{可能性 } \frac{1}{2} \\ 
U(-10, -5) & \text{可能性 } \frac{1}{4} 
\end{cases}
$$

In [45]:
# direct initialization
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0, 2] = 27

print(net[0].weight.data[0]) 
print(net[0].weight.data) 

tensor([42.,  6., 27., 29.])
tensor([[ 4.2000e+01,  6.0000e+00,  2.7000e+01,  2.9000e+01],
        [ 6.0000e+00,  6.0000e+00, -1.5281e+00, -3.5152e+00],
        [ 1.3968e-02,  1.1984e+01,  1.4727e+01, -2.1026e+00],
        [ 1.5819e+01,  6.0000e+00,  6.0000e+00,  6.0000e+00],
        [ 6.0000e+00,  6.0000e+00,  6.0000e+00,  6.0000e+00],
        [ 6.0000e+00,  1.4117e+01, -1.0386e+00,  6.0000e+00],
        [ 1.4862e+01,  6.0000e+00, -3.3229e+00, -3.0641e+00],
        [ 6.0000e+00,  6.0000e+00, -6.1504e-01,  1.4682e+01]])


In [51]:
# share a layer
shared_layer = nn.Linear(8,8) 
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), 
                    shared_layer, nn.ReLU(),
                    shared_layer, nn.ReLU(), 
                    nn.Linear(8,1)) 

# now we've define net[2] and net[5] as the same layer(shared_layer)
net(X) 
print(net[2].weight.data[0] == net[4].weight.data[0]) 
print(net[2].weight.data[0,0] )

net[2].weight.data[0,0] = 100 
print(net[2].weight.data[0] == net[4].weight.data[0])
print(net[2].weight.data[0,0] )

tensor([True, True, True, True, True, True, True, True])
tensor(0.2366)
tensor([True, True, True, True, True, True, True, True])
tensor(100.)


# 3. Custom layer

In [56]:
import torch
import torch.nn.functional as F
from torch import nn 

# define a layer without any params
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__() 
        
    def forward(self, X):
        return X - X.mean() 

layer = CenteredLayer()
print(layer(torch.FloatTensor([1, 2, 3, 4, 5]))) 

net = nn.Sequential(nn.Linear(8, 128), CenteredLayer()) 
Y = net(torch.rand(4, 8))
print(Y.mean())

tensor([-2., -1.,  0.,  1.,  2.])
tensor(-5.5879e-09, grad_fn=<MeanBackward0>)


In [60]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,)) # (units, ) this tuple only has one element
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear) 

linear = MyLinear(5, 3)
print(linear.weight)
print(linear(torch.rand(2, 5)))

net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
print(net(torch.rand(2, 64)))

Parameter containing:
tensor([[-0.9110, -1.1737,  0.8375],
        [-1.1390, -1.3665,  2.9099],
        [-0.4137, -0.2264,  0.4246],
        [-0.1646,  0.0253, -1.3546],
        [ 0.9051,  0.4764,  0.9985]], requires_grad=True)
tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.9762]])
tensor([[12.5350],
        [12.7621]])


# 4. Read & write file

In [66]:
import torch
from torch import nn
from torch.nn import functional as F
x = torch.arange(4)
torch.save(x, 'x-file') 
x2 = torch.load('x-file') # load file
print(x2)

tensor([0, 1, 2, 3])


C:\Users\admin\AppData\Local\Temp\ipykernel_34828\3155839136.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2 = torch.load('x-file') # load file


In [67]:
# we can also save a tensor list to the Memory 
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

C:\Users\admin\AppData\Local\Temp\ipykernel_34828\2061157211.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2, y2 = torch.load('x-files')


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [69]:
# we can also define a dict and display it as a more readable format
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

C:\Users\admin\AppData\Local\Temp\ipykernel_34828\1011335214.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mydict2 = torch.load('mydict')


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [74]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X) 

# save all the params of model
torch.save(net.state_dict(), 'mlp.params') 
clone = MLP() # Attention: the param is randomly initialize so it's different with the one we've saved
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

C:\Users\admin\AppData\Local\Temp\ipykernel_34828\1031783973.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clone.load_state_dict(torch.load('mlp.params'))


MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)